# Inconsistent Data Entry

## 환경설정

In [5]:
import pandas as pd
import numpy as np

# 텍스트 유사도(퍼지 매칭)
import fuzzywuzzy
from fuzzywuzzy import process

# 인코딩 추정 (이 노트북에서는 불러오기 정도만)
import charset_normalizer

professors = pd.read_csv("Data/pakistan_intellectual_capital.csv")
np.random.seed(0)

## 사전 전처리

In [7]:
# 데이터 훑어보기

professors.head()

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,Pakistan,2005.0,VLSI Electronics DLD Database,NaN


In [9]:
# Country column의 고유값을 확인해보니 대소문자, 공백, 오타 등의 입력 불일치가 보임

# Country column의 고유값 모두 불러오기
countries = professors['Country'].unique()
# 고유 값을 알파벳순으로 정렬
countries.sort()
countries

array([' Germany', ' New Zealand', ' Sweden', ' USA', 'Australia',
       'Austria', 'Canada', 'China', 'Finland', 'France', 'Greece',
       'HongKong', 'Ireland', 'Italy', 'Japan', 'Macau', 'Malaysia',
       'Mauritius', 'Netherland', 'New Zealand', 'Norway', 'Pakistan',
       'Portugal', 'Russian Federation', 'Saudi Arabia', 'Scotland',
       'Singapore', 'South Korea', 'SouthKorea', 'Spain', 'Sweden',
       'Thailand', 'Turkey', 'UK', 'USA', 'USofA', 'Urbana', 'germany'],
      dtype=object)

In [10]:
# 소문자화

professors['Country'] = professors['Country'].str.lower()

In [11]:
# 앞뒤 공백 제거

professors['Country'] = professors['Country'].str.strip()

## 퍼지 매칭(fuzzy matching)으로 남은 불일치 정리
- 데이터가 작으면 수동 수정도 가능하지만, 규모가 커지면 자동화 필요
- 퍼지 매칭은 목표 문자열과 편집 거리(문자 추가, 삭제, 교체 횟수)가 가까운 후보를 찾아줌

In [12]:
# Country column의 고유값 모두 불러오기
countries = professors['Country'].unique()
# 고유 값을 알파벳순으로 정렬
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea',
       'southkorea', 'spain', 'sweden', 'thailand', 'turkey', 'uk',
       'urbana', 'usa', 'usofa'], dtype=object)

In [14]:
# 'south korea'와 가장 가까운 10개 후보
# 'southkorea'가 유사도 48로 가깝게 나옴

matches = fuzzywuzzy.process.extract(
    "south korea",
    countries,
    limit=10,
    scorer=fuzzywuzzy.fuzz.token_sort_ratio
)
matches

[('south korea', 100),
 ('southkorea', 48),
 ('saudi arabia', 43),
 ('norway', 35),
 ('ireland', 33),
 ('portugal', 32),
 ('singapore', 30),
 ('netherland', 29),
 ('macau', 25),
 ('usofa', 25)]

In [15]:
# 유사도가 기본 47 이상인 항목들을 통일된 문자열로 치환하는 함수 만들기

def replace_matches_in_column(df, column, string_to_match, min_ratio=47):
    # 고유 문자열 목록
    strings = df[column].unique()

    # 후보 및 점수
    matches = fuzzywuzzy.process.extract(
        string_to_match, strings, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio
    )

    # 임계값 이상인 문자열만 추출
    close_matches = [m[0] for m in matches if m[1] >= min_ratio]

    # 해당 값들을 가진 행 찾기
    rows_with_matches = df[column].isin(close_matches)

    # 통일된 문자열로 치환
    df.loc[rows_with_matches, column] = string_to_match
    print("All done!")

In [16]:
# 해당 함수를 적용해 'southkorea'를 'south korea'로 모두 변경

replace_matches_in_column(
    df=professors, column='Country', string_to_match="south korea"
)

All done!


In [18]:
# 'south korea'만 남음

# Country column의 고유값 모두 불러오기
countries = professors['Country'].unique()
# 고유 값을 알파벳순으로 정렬
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea', 'spain',
       'sweden', 'thailand', 'turkey', 'uk', 'urbana', 'usa', 'usofa'],
      dtype=object)